In [2]:
import requests
from lxml import html
from bs4 import BeautifulSoup as bs
import pandas as pd


rickymorty = "https://www.imdb.com/title/tt2861424/episodes" #Link donde se buscará la información
encabezado = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
#Este encabezado puede ser genérico sin embargo hay que indagar si presenta modificaciones según caso de uso

datos = requests.get(rickymorty, headers= encabezado)

datos2 = html.fromstring(datos.text)
soup = bs(datos.text, "html.parser")
titulo = soup.find_all(class_= "sc-9115db22-7 sOSGg")
calificaciones = soup.find_all(class_="sc-e3e7b191-0 iKUUVe sc-f1a948e3-3 bQrxup")

# Itera a través de los elementos y obtén el texto
for title, calif in zip(titulo, calificaciones):
    #Dividir texto de calificaciones
    cap = title.text.split()[0].replace(".", "")
    cap_parts = cap.split("E")
    cap_parts[0] = "S" + cap_parts[0][1:].zfill(2)  # Agrega "0" después de la "S" si es necesario
    cap_parts[1] = "E" + cap_parts[1].zfill(2)  # Rellena con ceros a la izquierda de la E si es necesario
    cap = "".join(cap_parts)
    stars = calif.text.split()[0]
    votes = calif.text.split("(", 1)[1].split(")")[0]


    capituloycalificación = cap + "-" + stars + "-" + votes

    print(capituloycalificación)



In [5]:
base_url = "https://www.imdb.com/title/tt2861424/episodes"  # URL de la página principal
encabezado = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

capituloycalificación = []
# Itera a través de las páginas de episodios utilizando el segmento '?season='
# Esto sucede porque se está navegando con la etiqueta temporada de la página
season_number = 1
max_season = 5
while season_number <= max_season:
    # Construye la URL de la página actual y de navegación horizontal
    current_url = f"{base_url}/?season={season_number}"

    # Realiza una solicitud GET a la página actual
    response = requests.get(current_url, headers=encabezado)
    soup = bs(response.text, "html.parser")

    # Encuentra los elementos de título y calificación en la página actual
    titulo = soup.find_all(class_="sc-f2169d65-5 dUeItJ")
    calificaciones = soup.find_all(class_="sc-e2dbc1a3-0 ajrIH sc-282bae8e-3 bXuGWE")

    # Si no se encuentran elementos en la página, sal del bucle
    if not titulo:
        break

    # Itera a través de los elementos y obtén el texto
    for title, calif in zip(titulo, calificaciones):
        # Dividir texto de calificaciones
        cap = title.text.split()[0].replace(".", "") #Acceder al contenido del título, separandolo en espacios y reemplazando el punto entre 
                                                    #la abreviación y el nombre del título
        cap_parts = cap.split("E") #Dividir el texto del capítulo en partes buscando la E
        cap_parts[0] = "S" + cap_parts[0][1:].zfill(2)  # Agrega "0" después de la "S" si es necesario
        cap_parts[1] = "E" + cap_parts[1].zfill(2)  # El split elimina la E así que es necesario agregarlo nuevamente
                                                    # Y además Rellena con ceros a la izquierda de la E si es necesario
        cap = "".join(cap_parts) #Unir en un solo grupo las partes extraidas
        stars = calif.text.split()[0] #Acceder a la calificación obtenida a través la plataforma
        votes = calif.text.split("(", 1)[1].split(")")[0] #Acceder al dato de cantidad de votos obtenidos sin contemplar los paréntesis existentes
        capituloycalificación.append((cap, stars, votes))
        #print(capituloycalificación)

    # Incrementa el número de temporada para pasar a la siguiente página
    season_number += 1

rating = pd.DataFrame(capituloycalificación, columns=["Cap-Seasons", "Stars", "votes"])
print(rating)



   Cap-Seasons   Stars votes
0       S01E01  7.9/10   17K
1       S01E02  8.7/10   17K
2       S01E03  8.3/10   16K
3       S01E04  8.7/10   16K
4       S01E05  9.0/10   17K
5       S01E06  9.0/10   17K
6       S01E07  7.9/10   14K
7       S01E08  8.7/10   16K
8       S01E09  8.4/10   14K
9       S01E10  9.3/10   18K
10      S01E11  8.4/10   14K
11      S02E01  8.8/10   15K
12      S02E02  8.8/10   15K
13      S02E03  8.5/10   14K
14      S02E04  9.5/10   21K
15      S02E05  8.2/10   14K
16      S02E06  9.3/10   17K
17      S02E07  8.4/10   13K
18      S02E08  7.6/10   14K
19      S02E09  8.5/10   14K
20      S02E10  9.3/10   17K
21      S03E01  9.6/10   24K
22      S03E02  8.1/10   14K
23      S03E03  9.3/10   22K
24      S03E04  8.1/10   14K
25      S03E05  8.4/10   13K
26      S03E06  8.8/10   15K
27      S03E07  9.8/10   36K
28      S03E08  8.9/10   14K
29      S03E09  7.9/10   12K
30      S03E10  8.2/10   12K
31      S04E01  8.9/10   16K
32      S04E02  8.2/10   14K
33      S04E03